In [9]:
#AI chat bot project that utilizes a 3-layered neural network and natural language processing to gradually improve responses.
#Josh Lee
#07/28/2021
#Import libraries
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [10]:
#initialize variables, lemmatizer, and intents.json file
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!', ',', '.']


In [11]:
#create tokens for the patterns; use json module to load in file and save as variable 'intents'
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize words
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            

In [12]:
#lemmatize words; for example, working, works, worker all become work
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))
pickle.dump(words, open('words.pk1', 'wb'))
pickle.dump(classes, open('classes.pk1', 'wb'))

In [13]:
print(words)

["'s", 'Adverse', 'Awesome', 'Blood', 'Bye', 'Find', 'Give', 'Good', 'Goodbye', 'Hello', 'Hi', 'Hospital', 'How', 'I', 'ID', 'Is', 'List', 'Load', 'Locate', 'Looking', 'Lookup', 'Nice', 'Open', 'Search', 'Searching', 'See', 'Show', 'Take', 'Task', 'Thank', 'Thanks', 'That', 'Till', 'What', 'Which', 'Who', 'a', 'adverse', 'all', 'anyone', 'are', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'creator', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'for', 'have', 'help', 'helpful', 'helping', 'history', 'hospital', 'is', 'later', 'leader', 'list', 'log', 'lookup', 'made', 'management', 'me', 'module', 'nearby', 'next', 'of', 'offered', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'suitable', 'support', 'thanks', 'there', 'time', 'to', 'transfer', 'up', 'want', 'whats', 'with', 'you', 'your']


In [14]:
#beginning of training; matches words in the 'bag' with already known word patterns
training = []
output_empty = [0] * len(classes)
#put documents data into training list
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [15]:
#randomize training array, and split into x/y values
random.shuffle(training)
training = np.array(training, dtype=object)
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [16]:
#neural network of 3 layers, 128, 64, then number of intents
#multi-layer perception
#trained with stochastic gradient descent
model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))
sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 1)
model.save('chatbotmodel.h5', hist)

print('Done')

Epoch 1/200
10/10 [==============================] - 0s 1ms/step - loss: 2.3373 - accuracy: 0.0857
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.2290 - accuracy: 0.2391
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.2381 - accuracy: 0.1443
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.2300 - accuracy: 0.2075
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1472 - accuracy: 0.1842
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 1.9956 - accuracy: 0.4678
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8684 - accuracy: 0.4726
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7540 - accuracy: 0.4963
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7515 - accuracy: 0.6304
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.5133 - accuracy: 0.5369
Epoch 11/

10/10 [==============================] - 0s 2ms/step - loss: 0.1568 - accuracy: 0.9655
Epoch 83/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1222 - accuracy: 0.9782
Epoch 84/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1917 - accuracy: 0.9630
Epoch 85/200
10/10 [==============================] - 0s 3ms/step - loss: 0.1792 - accuracy: 0.9412
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1612 - accuracy: 0.9721
Epoch 87/200
10/10 [==============================] - 0s 2ms/step - loss: 0.2127 - accuracy: 0.9197
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0842 - accuracy: 0.9690
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1399 - accuracy: 0.9782
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1440 - accuracy: 0.9443
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1305 - accuracy: 0.9352
Epoch 92/200


10/10 [==============================] - 0s 1ms/step - loss: 0.1137 - accuracy: 0.9524
Epoch 164/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0966 - accuracy: 0.9412
Epoch 165/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0284 - accuracy: 0.9943
Epoch 166/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0287 - accuracy: 0.9943
Epoch 167/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0485 - accuracy: 0.9920
Epoch 168/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0584 - accuracy: 0.9943
Epoch 169/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0428 - accuracy: 0.9920
Epoch 170/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0518 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0964 - accuracy: 0.9443
Epoch 172/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0459 - accuracy: 0.9963
Epoc